In [ ]:
!pip install transformers[torch] -U
!pip install accelerate -U
!pip install datasets
!pip install evaluate
# from google.colab import files
# uploaded = files.upload()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 108.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.3
    Uninstalling transformers-4.46.3:
      Successfully uninstalled transformers-4.46.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.3/336.3 kB 23.7 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.1.1
    Uninstalling accelerate-1.1.1:
      Successfully uninstalled accelerate-1.1.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.2 MB/s eta 0:00:00
  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
train_file_path = "/content/drive/MyDrive/NLP/FinalProject/train.csv" #FILE PATH CHANGE IF NEEDED
val_file_path = "/content/drive/MyDrive/NLP/FinalProject/val.csv" #FILE PATH IF NEEDED

Mounted at /content/drive


In [ ]:
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from evaluate import load as load_metric
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import torch
from sklearn.metrics import accuracy_score, f1_score


In [ ]:
# Load datasets
train_data = pd.read_csv(train_file_path)
val_data = pd.read_csv(val_file_path)

# Encode genres into numeric labels
label_encoder = LabelEncoder()
train_data["label"] = label_encoder.fit_transform(train_data["Genre0"])
val_data["label"] = label_encoder.transform(val_data["Genre0"])

# Prepare datasets for Huggingface
train_dataset = Dataset.from_pandas(train_data[["Lyric", "label"]])
val_dataset = Dataset.from_pandas(val_data[["Lyric", "label"]])

dataset = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset
})
print(dict(enumerate(label_encoder.classes_)))


{0: 'Dance', 1: 'Heavy Metal', 2: 'Hip Hop', 3: 'Indie', 4: 'Pop', 5: 'Rock'}


In [ ]:
# Load pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenize the lyrics
def preprocess_function(examples):
    return tokenizer(examples["Lyric"], truncation=True, padding=True, max_length=512)

tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Load pre-trained model with classification head
num_labels = len(label_encoder.classes_)
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_labels)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/48493 [00:00<?, ? examples/s]

Map:   0%|          | 0/5389 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Define metrics for evaluation
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="weighted")
    return {"accuracy": acc, "f1": f1}

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/NLP/FinalProj/Model_Test_Final/",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print("Evaluation results:", results)

<ipython-input-8-05a79dfd0b8d>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.959400,0.969250,0.627760,0.574264


Evaluation results: {'eval_loss': 0.969249963760376, 'eval_accuracy': 0.6277602523659306, 'eval_f1': 0.5742639390503201, 'eval_runtime': 82.4323, 'eval_samples_per_second': 65.375, 'eval_steps_per_second': 4.088, 'epoch': 1.0}


In [ ]:
model.save_pretrained("/content/drive/MyDrive/NLP/genre_classification_model")
tokenizer.save_pretrained("/content/drive/MyDrive/NLP/genre_classification_model")

('/content/drive/MyDrive/NLP/genre_classification_model/tokenizer_config.json',
 '/content/drive/MyDrive/NLP/genre_classification_model/special_tokens_map.json',
 '/content/drive/MyDrive/NLP/genre_classification_model/vocab.txt',
 '/content/drive/MyDrive/NLP/genre_classification_model/added_tokens.json',
 '/content/drive/MyDrive/NLP/genre_classification_model/tokenizer.json')

In [ ]:
checkpoint_path = "/content/drive/MyDrive/NLP/FinalProj/Model_Test_Final/checkpoint-3031"

tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)

model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path)

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/NLP/FinalProj/Model_Test_Final",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,  # Ensure the total number of epochs is at least 2
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

# Reload your dataset (if not already loaded)
dataset = DatasetDict({
    "train": tokenized_datasets["train"],
    "validation": tokenized_datasets["validation"]
})

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=tokenizer
)

# Resume training from the specified checkpoint
trainer.train(resume_from_checkpoint=checkpoint_path)

# Evaluate the model after completing the second epoch
results = trainer.evaluate()
print("Evaluation results:", results)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-9-c50971566ea0>:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3418: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary obje

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3081: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint_rng_state = torch.load(rng_file)


Epoch,Training Loss,Validation Loss
2,0.831500,0.934036


KeyError: "The `metric_for_best_model` training argument is set to 'eval_accuracy', which is not found in the evaluation metrics. The available evaluation metrics are: ['eval_loss']. Consider changing the `metric_for_best_model` via the TrainingArguments."

In [ ]:
results = trainer.evaluate()
print("Evaluation results:", results)

Epoch,Training Loss,Validation Loss
2,0.831500,0.934036


Evaluation results: {'eval_loss': 0.9340356588363647}


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,  # Use the trained model
    args=training_args,  # Use the same training arguments
    eval_dataset=dataset["validation"],  # Validation dataset
    tokenizer=tokenizer,  # Tokenizer
    compute_metrics=compute_metrics  # Add compute_metrics
)
results = trainer.evaluate()
print("Evaluation results:", results)


<ipython-input-12-716cf97ac023>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Evaluation results: {'eval_loss': 0.9340356588363647, 'eval_model_preparation_time': 0.0016, 'eval_accuracy': 0.6405641120801633, 'eval_f1': 0.6076551710945605, 'eval_runtime': 83.4827, 'eval_samples_per_second': 64.552, 'eval_steps_per_second': 4.037}
